In [248]:
import random
import torch
import numpy as np
import math

In [116]:
import itertools
from abc import ABC, abstractmethod
from typing import Any, Dict, Iterable, List, Tuple

In [30]:
from cirkit.region_graph.rg_node import RGNode, RegionNode, PartitionNode
from cirkit.region_graph.region_graph import RegionGraph

# Symbolic representation

In [398]:
class SymbLayer(ABC): 
    inputs: List[ABC]
    outputs: List[ABC]
    scope: Iterable[int]
    layer_type: int 
    weight_placeholder: torch.Tensor
    output_placeholder: torch.Tensor
    
    real_weights_pointer: torch.Tensor
    
    product_operation: Tuple
    
    def __init__(self, layer_type, scope, hidden_dim=None, output_dim=None, weight=None): 
        assert (layer_type in [0,1,2])
        self.layer_type = layer_type
        self.scope = scope
        
        if weight is not None: 
            assert (layer_type == 0)
            self.output_placeholder = torch.empty(weight.shape[0]) 
            self.weight_placeholder = torch.empty((weight.shape[0], weight.shape[1])) 
            
        if hidden_dim is not None: 
            if layer_type == 0: 
                assert (output_dim is not None)
                self.output_placeholder = torch.empty(output_dim) 
                self.weight_placeholder = torch.empty((output_dim, hidden_dim**2)) 
            elif layer_type == 1: 
                self.output_placeholder = torch.empty(hidden_dim**2)
            elif layer_type == 2: 
                self.output_placeholder = torch.empty(hidden_dim)
        
        self.inputs = []
        self.outputs = []
        
    def set_product_operation(self, product_operation): 
        self.product_operation = product_operation 
        
    def set_real_weights_pointer(self, real_weights_pointer): 
        self.real_weights_pointer = real_weights_pointer
    
    @property 
    def is_sum(self): 
        return self.layer_type == 0
    @property
    def is_product(self): 
        return self.layer_type == 1
    @property
    def is_leaf(self): 
        return self.layer_type == 2
    
# actually not used 
class SymbGraph(): 
    def __init__(self): 
        self._layers: Set[SymbLayer] = set()
            
    def add_layer(self, layer: SymbLayer): 
        self._layers.add(layer)
        
    def add_edge(self, tail: SymbLayer, head: SymbLayer):
        self._layers.add(tail)
        self._layers.add(head)
        tail.outputs.append(head)
        head.inputs.append(tail)


# Build 2 PC in symbolic represenatation, 
# s_graph with hidden_dim = 3; 
# s_graph_2 with hidden_dim = 5; 
# Both with the same structure but different scope ([1,2,3,4] and [3,4,5,6]) 

In [399]:
s_graph = SymbGraph()

graph_1_hidden_dim = 3

n1 = SymbLayer(2, [1], graph_1_hidden_dim)
n2 = SymbLayer(2, [2], graph_1_hidden_dim)
np1 = SymbLayer(1, [1,2], graph_1_hidden_dim)
ns1 = SymbLayer(0, [1,2], graph_1_hidden_dim, graph_1_hidden_dim)

n3 = SymbLayer(2, [3], graph_1_hidden_dim)
n4 = SymbLayer(2, [4], graph_1_hidden_dim)
np2 = SymbLayer(1, [3,4], graph_1_hidden_dim)
ns2 = SymbLayer(0, [3,4], graph_1_hidden_dim, graph_1_hidden_dim) 

np3 = SymbLayer(1, [1,2,3,4], graph_1_hidden_dim)
ns3 = SymbLayer(0, [1,2,3,4], graph_1_hidden_dim, 1)

s_graph.add_edge(n1, np1)
s_graph.add_edge(n2, np1)
s_graph.add_edge(np1, ns1)

s_graph.add_edge(n3, np2)
s_graph.add_edge(n4, np2)
s_graph.add_edge(np2, ns2) 

s_graph.add_edge(ns1, np3)
s_graph.add_edge(ns2, np3)
s_graph.add_edge(np3, ns3) 

# PC2 

s_graph_2 = SymbGraph()

graph_2_hidden_dim = 5

n1_2 = SymbLayer(2, [5], graph_2_hidden_dim)
n2_2 = SymbLayer(2, [6], graph_2_hidden_dim)
np1_2 = SymbLayer(1, [5,6], graph_2_hidden_dim)
ns1_2 = SymbLayer(0, [5,6], graph_2_hidden_dim, graph_2_hidden_dim)

n3_2 = SymbLayer(2, [3], graph_2_hidden_dim)
n4_2 = SymbLayer(2, [4], graph_2_hidden_dim)
np2_2 = SymbLayer(1, [3,4], graph_2_hidden_dim)
ns2_2 = SymbLayer(0, [3,4], graph_2_hidden_dim, graph_2_hidden_dim) 

np3_2 = SymbLayer(1, [3,4,5,6], graph_2_hidden_dim)
ns3_2 = SymbLayer(0, [3,4,5,6], graph_2_hidden_dim, 1)

s_graph_2.add_edge(n1_2, np1_2)
s_graph_2.add_edge(n2_2, np1_2)
s_graph_2.add_edge(np1_2, ns1_2)

s_graph_2.add_edge(n3_2, np2_2)
s_graph_2.add_edge(n4_2, np2_2)
s_graph_2.add_edge(np2_2, ns2_2) 

s_graph_2.add_edge(ns1_2, np3_2)
s_graph_2.add_edge(ns2_2, np3_2)
s_graph_2.add_edge(np3_2, ns3_2) 



# set parameters of the two folded PCs, 
# First PC: Q_1 and WV_1; second PC: Q_2 and WV_2
# Then in each sum node of the unfolded symbolic representation, points to the corresponding slice of the folded parammeters

# this is to emulate [folded tensorized circuit -> symbolic representation]

In [416]:
from cirkit.reparams.leaf import ReparamExp
reparam = ReparamExp
# num_folds, output_dim, input_dim 
Q_1 = reparam((1, 1, graph_1_hidden_dim**2), dim=2)
WV_1 = reparam((2, graph_1_hidden_dim, graph_1_hidden_dim**2), dim=2)

Q_2 = reparam((1, 1, graph_2_hidden_dim**2), dim=2)
WV_2 = reparam((2, graph_2_hidden_dim, graph_2_hidden_dim**2), dim=2)

ns3.set_real_weights_pointer(Q_1.param[0])
ns1.set_real_weights_pointer(WV_1.param[0])
ns2.set_real_weights_pointer(WV_1.param[1])

ns3_2.set_real_weights_pointer(Q_2.param[0])
ns1_2.set_real_weights_pointer(WV_2.param[0])
ns2_2.set_real_weights_pointer(WV_2.param[1])


# Do the product with symbolic representation, following the pseudocode 
# but ignoring mixing layer, or different layer types (between layer1 and layer2)

# considered sub-PC with distinct scope here, used function match_dim to pad the sub-PC with distinct scope to the dimension of circuit product

In [467]:
def symbolic_product(layer1, layer2): 
    
    M = layer1.output_placeholder.shape[0]
    N = layer2.output_placeholder.shape[0]
    
    if not set(layer1.scope).intersection(layer2.scope): 
        assert (layer1.is_sum or layer1.is_leaf) 
        assert (layer2.is_sum or layer2.is_leaf)  
        
        new_layer_1 = match_dim(layer1, N)
        new_layer_2 = match_dim(layer2, M)
        
        new_scope = list(set(new_layer_1.scope) | set(new_layer_2.scope))
        
        new_product_layer = SymbLayer(1, new_scope, hidden_dim=M*N)
        
        product_operation = ('diff_scope_product', layer1, layer2)
        new_product_layer.set_product_operation(product_operation)

        new_layer_1.outputs.append(new_product_layer)
        new_layer_2.outputs.append(new_product_layer)
        new_product_layer.inputs.append(new_layer_1)
        new_product_layer.inputs.append(new_layer_2)
        
        pseudo_sum_layer = SymbLayer(0, new_scope, hidden_dim=M*N, output_dim=M*N) 
        
        product_operation = ('diff_scope_identity_sum', layer1, layer2)
        pseudo_sum_layer.set_product_operation(product_operation)
        
        new_product_layer.outputs.append(pseudo_sum_layer)
        pseudo_sum_layer.inputs.append(new_product_layer)
        
        return pseudo_sum_layer 
        
        
        
        # product, then "identity sum", but also tweak the vector length (zero padding) 
    
    elif layer1.is_leaf and layer2.is_leaf: 
        assert (layer1.scope == layer2.scope)
        
        new_layer = SymbLayer(2, layer1.scope, M*N) 
        product_operation = ('input_product', layer1, layer2)
        new_layer.set_product_operation(product_operation)
        return new_layer
        
    elif layer1.is_sum and layer2.is_sum: 
        layer1_input = layer1.inputs 
        layer2_input = layer2.inputs 
        
        assert (len(layer1_input) == 1)
        assert (len(layer2_input) == 1)
        
        new_layer_input = symbolic_product(layer1_input[0], layer2_input[0])
        
        layer1_weight_placeholder = layer1.weight_placeholder
        layer2_weight_placeholder = layer2.weight_placeholder
        
        new_layer_weight_placeholder = np.kron(layer1_weight_placeholder, layer2_weight_placeholder) 
    
        new_scope = list(set(layer1.scope) | set(layer2.scope))
        
        new_layer = SymbLayer(0, new_scope, weight=new_layer_weight_placeholder)
        
        product_operation = ('sum_product', layer1, layer2)
        new_layer.set_product_operation(product_operation)
        
        new_layer_input.outputs.append(new_layer)
        new_layer.inputs.append(new_layer_input)
        
        return new_layer
    
    elif layer1.is_product and layer2.is_product: 
        layer1_input = layer1.inputs 
        layer2_input = layer2.inputs 
        
        assert (len(layer1_input) == 2)
        assert (len(layer2_input) == 2)
        
        new_layer_input_left = symbolic_product(layer1_input[0], layer2_input[0])
        new_layer_input_right = symbolic_product(layer1_input[1], layer2_input[1])
        
        new_scope = list(set(layer1.scope) | set(layer2.scope)) 
        
        new_layer = SymbLayer(1, new_scope, hidden_dim=int(math.sqrt(M*N))) # hack with sqrt 
        
        product_operation = ('product_product', layer1, layer2)
        new_layer.set_product_operation(product_operation)
        
        new_layer_input_left.outputs.append(new_layer)
        new_layer_input_right.outputs.append(new_layer)
        new_layer.inputs.append(new_layer_input_left)
        new_layer.inputs.append(new_layer_input_right)
        
        return new_layer 
        
        
        
def match_dim(layer, N): 
    
    M = layer.output_placeholder.shape[0]
    
    if layer.is_leaf: 
        
        new_layer = SymbLayer(2, layer.scope, M*N) 
        product_operation = ('diff_scope_input_pad', layer) 
        new_layer.set_product_operation(product_operation) 
        return new_layer
   
    elif layer.is_sum: 
        
        new_layer_input = match_dim( layer.inputs[0], N )
        
        new_layer = SymbLayer(0, layer.scope, hidden_dim=M*N, output_dim=M*N) # hack 
        
        product_operation = ('diff_scope_sum_pad', layer)
        new_layer.set_product_operation(product_operation)
        
        new_layer_input.outputs.append(new_layer)
        new_layer.inputs.append(new_layer_input)
        
        return new_layer
        
        
        
        
    elif layer.is_product: 
        
        new_layer_input_left = match_dim( layer.inputs[0], N )
        new_layer_input_right = match_dim( layer.inputs[1], N )
        
        new_layer = SymbLayer(1, layer.scope, hidden_dim=M*N) 
        
        product_operation = ('diff_scope_product', layer)
        new_layer.set_product_operation(product_operation)
        
        new_layer_input_left.outputs.append(new_layer)
        new_layer_input_right.outputs.append(new_layer)
        new_layer.inputs.append(new_layer_input_left)
        new_layer.inputs.append(new_layer_input_right)
        
        return new_layer
        
    
    

# do symbolic product, and feel free to trace into the input of each node to examinate the structure

In [417]:
symb_product_result = symbolic_product(ns3, ns3_2)

In [424]:
symb_product_result.__dict__['inputs'][0].__dict__['inputs'][0].__dict__
symb_product_result.__dict__['inputs'][0].__dict__['inputs'][1].__dict__
# symb_product_result.__dict__['inputs'][0].__dict__['inputs']

# HAVE PARAMS
# symb_product_result.__dict__['inputs'][0].__dict__['inputs'][1].__dict__['product_operation'][1].__dict__
# symb_product_result.__dict__['inputs'][0].__dict__['inputs'][1].__dict__['product_operation'][2].__dict__

symb_product_result.__dict__['inputs'][0].__dict__['inputs'][0].__dict__['inputs'][0].__dict__

{'layer_type': 1,
 'scope': [1, 2, 5, 6],
 'output_placeholder': tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -5.4223e-05,
          4.5850e-41, -5.4223e-05,  4.5850e-41,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00, 

# This part is to emulate the step to build folded tensorized PC from symbolic representation of a circuit product 

# got topological_layer_result after topological sort 

# it is not using the parameters in TuckerLayer(). Instead, it is using params. 

# By constructing the parameters in each fold and concatenating (appending) each fold 

# Condition: if the current fold is "product_operation", or "diff_scope_identity_sum", or "diff_scope_sum_pad"

In [456]:
from cirkit.layers.sum_product.tucker import TuckerLayer

# AFTER topological sort, 

symb_sum_1 = symb_product_result.__dict__['inputs'][0].__dict__['inputs'][0]
symb_product_1 = symb_sum_1.__dict__['inputs'][0]

symb_sum_2 = symb_product_result.__dict__['inputs'][0].__dict__['inputs'][1]
symb_product_2 = symb_sum_2.__dict__['inputs'][0]

topological_layer_result = [(symb_product_1, symb_product_2), (symb_sum_1, symb_sum_2)]

shape_of_input_to_product = topological_layer_result[0][1].__dict__['inputs'][0].__dict__['output_placeholder'].shape[0]
shape_of_output = topological_layer_result[1][0].__dict__['output_placeholder'].shape[0]

layer = TuckerLayer(shape_of_input_to_product, shape_of_output, arity=2, num_folds=len(topological_layer_result[0]))

params = [] 

for symb_sum in topological_layer_result[1]: 
    product_operation = symb_sum.__dict__['product_operation']
    if product_operation[0] == 'sum_product': 
        param_product_1 = product_operation[1].__dict__['real_weights_pointer'] 
        param_product_2 = product_operation[2].__dict__['real_weights_pointer'] 
        params.append(torch.kron(param_product_1, param_product_2)  ) 
        
    elif product_operation[0] == 'diff_scope_identity_sum': 
        rows = symb_sum.__dict__['weight_placeholder'].shape[0]
        cols = symb_sum.__dict__['weight_placeholder'].shape[1] 
        
        # set gradient of this particular fold to 0 during gradient update 
        identity_sum = np.hstack((np.eye(rows), np.zeros((rows, cols - rows)))) # actually need to preturb, not just identity + zeros
        params.append(identity_sum) 
        
    elif product_operation[0] == 'diff_scope_sum_pad': 
        weight_original = product_operation[1].__dict__['real_weights_pointer']
        
        shape_product = symb_sum['weight_placeholder'].shape
        shape_original = weight_original.shape
        
        shape_other = shape_product[0]/shape_original[0] # hack
        
        # set gradient of this particular component to 0 during gradient update 
        padding_matrix = np.concatenate( ( np.ones ((1,shape_other**2)) , np.zeros ( (shape_other-1,shape_other**2)) ) , axis=0)
        
        padded_sum_weight = torch.kron(padding_matrix, weight_original)
        params.append(padded_sum_weight) 
        
        
params = torch.tensor(params)    
    






In [464]:
torch.tensor(params)

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., -0., 0.,  ..., 0., -0., 0.],
         [-0., 0., -0.,  ..., -0., 0., -0.],
         [0., -0., 0.,  ..., 0., -0., 0.],
         ...,
         [0., -0., 0.,  ..., 0., -0., 0.],
         [-0., 0., -0.,  ..., -0., 0., -0.],
         [0., -0., 0.,  ..., 0., -0., 0.]]], dtype=torch.float64)

# pad the sub-PC with distinct scope: 


In [470]:
M=2
N=3 

leaf_left = np.random.rand(2)
leaf_right = np.random.rand(2)
weights_original = np.random.rand(2,4)

In [474]:
leaf_left_pad = np.pad(leaf_left, ((0, M*N-M)), 'constant')
leaf_right_pad = np.pad(leaf_right, ((0, M*N-M)), 'constant')

identity_pad = np.concatenate( ( np.ones((1,N**2)), np.zeros((N-1,N**2)) ) , axis=0)

weights_pad = np.kron(identity_pad, weights_original)
                    # first weight dim = M*(M^2), second weight dim = N*(N^2)
result = np.matmul(weights_pad, np.kron(leaf_left_pad, leaf_right_pad) ) 

original_result = np.matmul(weights_original, np.kron(leaf_left, leaf_right))

assert (np.linalg.norm(result[:2] - original_result) < 1e-7)

result

array([0.91967042, 0.78356915, 0.        , 0.        , 0.        ,
       0.        ])

In [348]:
# n_test_1 = SymbLayer(2, [1], 3)
# n_test_2 = SymbLayer(2, [2], 5)

# np_test_1 = SymbLayer(1, [1,2], 3)
# ns_test_1 = SymbLayer(0, [1,2], 3, 3)

# n_test_1.outputs.append(np_test_1)
# n_test_2.outputs.append(np_test_1)
# np_test_1.inputs.append(n_test_1)
# np_test_1.inputs.append(n_test_2)

# np_test_1.outputs.append(ns_test_1)
# ns_test_1.inputs.append(np_test_1)

# n_test_3 = SymbLayer(2, [3], 5)
# n_test_4 = SymbLayer(2, [4], 5)

# np_test_2 = SymbLayer(1, [3,4], 5)

# ns_test_2 = SymbLayer(0, [3,4], 5, 5)

# n_test_3.outputs.append(np_test_2)
# n_test_4.outputs.append(np_test_2)
# np_test_2.inputs.append(n_test_3)
# np_test_2.inputs.append(n_test_4)

# np_test_2.outputs.append(ns_test_2)
# ns_test_2.inputs.append(np_test_2)


# symbolic_product(ns_test_1, ns_test_2).__dict__['inputs'][0].__dict__['inputs'][0].__dict__['inputs'][0].__dict__['inputs'][0].__dict__


# # .__dict__['inputs'][0].__dict__['inputs'][1].__dict__['product_operation'][-1].__dict__ 







{'layer_type': 2,
 'scope': [1],
 'output_placeholder': tensor([-9.1859e-11,  4.5850e-41, -9.5739e-13,  4.5850e-41, -3.7259e-10,
          4.5850e-41, -3.4776e-10,  4.5850e-41, -7.9532e-11,  4.5850e-41,
         -9.5709e-13,  4.5850e-41, -3.7247e-10,  4.5850e-41, -3.7259e-10]),
 'inputs': [],
 'outputs': [<__main__.SymbLayer at 0x7fd0b8895520>],
 'product_operation': ('diff_scope_input_pad',
  <__main__.SymbLayer at 0x7fd0b86be2b0>)}